In [25]:
from transformers import pipeline, AutoTokenizer, BertForTokenClassification

noktalama_isaretleri = ['!', '?', '.', ',', '-', ':', ';', "'"]

def preprocess(text):
    # Define punctuation marks to keep
    noktalama_isaretleri = ['!', '?', '.', ',', '-', ':', ';', "'"]
    # Remove punctuation marks defined in noktalama_isaretleri
    new_text = ''.join(char for char in text if char not in noktalama_isaretleri)
    print("New text: ", new_text)
    
    # # First pass: Keep alphanumeric, spaces and punctuation marks
    # new_text = ""
    # for char in text:
    #     if (char in noktalama_isaretleri or 
    #         char.isalnum() or 
    #         char.isspace()):
    #         new_text += char
            
    # # Second pass: Keep only alphanumeric, spaces and specific marks
    # new_text_pure = ""
    # for char in text:
    #     if (char.isalnum() or 
    #         char.isspace() or 
    #         char in ["'", "-"]):
    #         new_text_pure += char
            
    # # Replace special characters with spaces
    # new_text_pure = new_text_pure.replace("'", " ")
    # new_text_pure = new_text_pure.replace("-", " ")
    
    # # Convert to lowercase and handle Turkish characters
    # new_text = new_text_pure.replace("I", "ı").lower()
    return new_text


def punctate(sent, punc_corr):
    sent = preprocess(sent)
    r2 = punc_corr(sent)
    for r in r2:
        print(r)
    tokenized_sent = tokenizer.tokenize(sent)
    final_sent = ''

    i = 0

    while i < len(tokenized_sent):
        token = tokenized_sent[i]

        if r2[i]['entity'] != 'non':
            token += r2[i]['entity']

        if r2[i]['entity'] not in ["'", "-"]:
            token += ' '

        final_sent += token
        i += 1

    final_sent = final_sent.replace(' ##', '')

    return final_sent


def end2end(sent, capitalization_corr, punc_corr):

    p_sent = preprocess(sent)

    r1 = capitalization_corr(p_sent)
    r2 = punc_corr(p_sent)
    print("Punc: ", r2)

    tokenized_sent = tokenizer.tokenize(p_sent)

    final_sent = ''

    i = 0

    while i < len(tokenized_sent):
        token = tokenized_sent[i]
        if r1[i]['entity'] == 'one':
            token = token.capitalize()
        elif r1[i]['entity'] == 'cap':
            token = token.upper()
            while tokenized_sent[i+1].startswith("##"):
                token += tokenized_sent[i+1][2:].upper()
                i += 1

        if r2[i]['entity'] != 'non':
            token += r2[i]['entity']

        if r2[i]['entity'] not in ["'", "-"]:
            token += ' '

        final_sent += token
        i += 1

    final_sent = final_sent.replace(' ##', '')

    return final_sent


cap_model = BertForTokenClassification.from_pretrained(
    "ytu-ce-cosmos/turkish-base-bert-capitalization-correction")
punc_model = BertForTokenClassification.from_pretrained(
    "ytu-ce-cosmos/turkish-base-bert-punctuation-correction")

tokenizer = AutoTokenizer.from_pretrained(
    "ytu-ce-cosmos/turkish-base-bert-capitalization-correction")


capitalization_corr = pipeline("ner", model=cap_model, tokenizer=tokenizer)
punc_corr = pipeline("ner", model=punc_model, tokenizer=tokenizer)

sent = """
Tabelaları çok küçük, renksiz, ışıksız. Gerçekten anlayamıyoruz [pp] (fdfd).
"""

# print(end2end(sent, capitalization_corr, punc_corr))
res = punctate(sent, punc_corr)
# Geçen hafta sonu arkadaşlarımla birlikte kısa bir tatile çıktık. Cuma akşamı yola çıktık. Yolculuk oldukça keyifli geçti. Cumartesi sabahı otele vardık. Odalarımıza yerleştikten sonra kahvaltıya indik. Kahvaltıda birçok seçenek vardı; omlet, simit, taze sıkılmış portakal suyu ve çeşitli peynirler. Kahvaltıdan sonra sahile gitmeye karar verdik. Deniz çok sakindi ve hava mükemmeldi. Denizde yüzdük, kumda yürüdük ve güneşlendik. Öğleden sonra şehri gezmeye çıktık. Tarihi yerleri ziyaret ettik ve bol bol fotoğraf çektik. Akşam yemeği için meşhur bir restorana gittik. Deniz ürünleri gerçekten çok tazeydi. Yemek sonrası otele döndüğümüzde çok yorgunduk ama tatilin ilk günü harika geçmişti. Pazar sabahı erken kalkıp bir doğa yürüyüşüne çıktık. Orman içinde yürümek çok huzur vericiydi. Dönüş yolunda biraz trafik vardı ama bu güzel tatilin ardından hiçbiri moralimizi bozamazdı. Eve vardığımızda herkes mutlu ve huzurluydu. Bir sonraki tatili planlamaya başladık bile.


New text:  
Tabelaları çok küçük renksiz ışıksız Gerçekten anlayamıyoruz [pp] (fdfd)

{'entity': 'non', 'score': 0.9999956, 'index': 1, 'word': 'Tabelaları', 'start': 1, 'end': 11}
{'entity': 'non', 'score': 1.0, 'index': 2, 'word': 'çok', 'start': 12, 'end': 15}
{'entity': ',', 'score': 0.99981433, 'index': 3, 'word': 'küçük', 'start': 16, 'end': 21}
{'entity': 'non', 'score': 1.0, 'index': 4, 'word': 'renk', 'start': 22, 'end': 26}
{'entity': ',', 'score': 0.9999659, 'index': 5, 'word': '##siz', 'start': 26, 'end': 29}
{'entity': 'non', 'score': 1.0, 'index': 6, 'word': 'ışık', 'start': 30, 'end': 34}
{'entity': 'non', 'score': 0.9354928, 'index': 7, 'word': '##sız', 'start': 34, 'end': 37}
{'entity': 'non', 'score': 0.9999982, 'index': 8, 'word': 'Gerçekten', 'start': 38, 'end': 47}
{'entity': 'non', 'score': 1.0, 'index': 9, 'word': 'anlaya', 'start': 48, 'end': 54}
{'entity': 'non', 'score': 0.6378202, 'index': 10, 'word': '##mıyoruz', 'start': 54, 'end': 61}
{'entity': 'non', 'sc

In [2]:
sent = """
Dördüncü dersimiz, arkadaşlar ve eğlenceyi hayatından çıkarma
"""

print(end2end(sent, capitalization_corr, punc_corr))

Dördüncü dersimiz arkadaşlar ve eğlenceyi hayatından çıkarma. 
